# 02 — Where is pip install? (Libraries in C++) 📦

In Python, if you want a library, you type:
bash
pip install numpy

And then in your code:
python
import numpy


In C++, there is no official pip. This is one of the biggest culture shocks for new C++ developers.

In this notebook, we will explain how C++ includes work and the three ways to manage dependencies in 2024.

---

## 1. How #include Actually Works

When you write #include <vector>, the Preprocessor (the first step of compilation) literally copy-pastes the contents of the vector file into your file.

### Angle Brackets < > vs Quotes " "

You will see both. The difference is where the compiler looks first.

* #include \<header\>: The compiler searches in the System Include Path (e.g., /usr/include, /usr/local/include). Used for Standard Library and installed 3rd party libs.
* #include "header.hpp": The compiler searches in the Current Directory first, then falls back to system paths. Used for your own project files.

### The Search Path (-I)

The compiler doesn't magically know where files are. You must tell it using the -I (Include) flag. 

If you have include/math_utils.hpp, you compile with -I./include. In CMake, this is handled by include_directories().

In [2]:
// Interactive Check: Where is iostream?
// We can use the preprocessor command #line to see file locations (system specific)
#include <iostream>

int main() {
    std::cout << "If this runs, the compiler found <iostream> in the system path!" << std::endl;
    return 0;
}

main();

If this runs, the compiler found <iostream> in the system path!


---

## 2. Managing External Libraries

So, if there is no pip, how do we get libraries like fmt (for formatting) or nlohmann/json (for JSON)?

We have 3 strategies, ranging from "Old School" to "Modern".

### Strategy A: The System Package Manager (Linux Style)
You rely on your OS.
bash
sudo apt-get install libfmt-dev

Then in CMake:

```cmake
find_package(fmt REQUIRED)
target_link_libraries(myapp fmt::fmt)
```

* Pros: Easy on Linux.
* Cons: Nightmare on Windows; you don't control the version (you get whatever Ubuntu gives you).

### Strategy B: Vendoring (git submodule)
You literally download the library source code into a libs/ folder in your project.
* Pros: You control everything. Zero external dependencies.
* Cons: Your repository gets huge. You have to manually update libraries.

### Strategy C: Modern CMake (FetchContent)
This is the modern standard for small to medium projects. CMake can download libraries while compiling.

Example CMakeLists.txt:
```cmake
include(FetchContent)

FetchContent_Declare(
  fmt
  GIT_REPOSITORY [https://github.com/fmtlib/fmt](https://github.com/fmtlib/fmt)
  GIT_TAG        10.1.0
)

FetchContent_MakeAvailable(fmt)

add_executable(app main.cpp)
target_link_libraries(app fmt::fmt)
```

### Strategy D: The "New" Package Managers (Conan / vcpkg)
These are the closest things to pip. They are command-line tools that install libraries into a central cache.

```bash
vcpkg install fmt
```

These are great for large enterprise projects, but FetchContent (Strategy C) is often enough for personal projects.

---

## 3. Summary

1.  Header (.hpp): Tells the compiler what exists (Declarations). Needs -I path.
2.  Library (.lib / .a): Contains the compiled code (Definitions). Needs Linker instructions.
3.  Dependency Management: 
    * Avoid manual downloading.
    * Use CMake FetchContent for simple projects.
    * Use vcpkg or Conan for complex projects.